In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import re

In [ ]:
fft_step   = 12.5/1000. # 12.5ms

audio_filenames = [ './librivox/guidetomen_%02d_rowland_64kb.mp3' % (i,) for i in [1,2,3]]
audio_filenames

mel_filenames = [ f.replace('.mp3', '.melspectra.hkl') for f in audio_filenames ]

In [ ]:
mel_filename_test = mel_filenames[1]

with open(mel_filename_test.replace('.hkl', '.16_k2.sym'), 'rt') as f:
    mel_sym_str = f.read()
    
mel_sym_chars = set(mel_sym_str)
mel_sym_dict  = { c:i for i,c in enumerate(list(mel_sym_chars)) }
mel_sym = np.array( [mel_sym_dict[c] for c in mel_sym_str] )

mel_sym_silence = mel_sym_dict[' ']

mel_sym_str[100:115], mel_sym[100:115], mel_sym.shape[0]

In [ ]:
# Compress the sym data, by counting the duplicates, 
# and storing 'initial char', 'char count' and 'initial char idx'
mel_sym_ct, mel_sym_cc, mel_sym_cn = [], [], []
prev_c, prev_n = '', 0
for t, c in enumerate(mel_sym_str):
    if c==prev_c: 
        prev_n+=1 # Add one to count
    else:
        mel_sym_cn.append(prev_n)  # Store count of previous char
        mel_sym_ct.append(t)       # Start on new char's index
        mel_sym_cc.append(c)       # Start on new char's value
        prev_c, prev_n = c, 1
mel_sym_cn.append(prev_n)  # Store last count value

mel_sym_ct = np.array( mel_sym_ct )
mel_sym_ci = np.array( [mel_sym_dict[c] for c in mel_sym_cc] )
mel_sym_cn = np.array( mel_sym_cn[1:])   # Kill the first value, and convert to numpy

(mel_sym_str[66:95], 
 mel_sym_cc[0:10], mel_sym_ci[0:10], 
 mel_sym_cn[0:10], mel_sym_ct[0:10], 
 mel_sym_ci.shape[0],)

In [ ]:
def print_one_sec_per_line(s, t_min=0., t_max=None):
    each_line = int(1/fft_step)
    if t_max is None: t_max = len(s)*fft_step
    for t in np.arange(t_min, t_max, 1.):
        i = int(t/fft_step)
        print( s[i:i+each_line] )
        
print_one_sec_per_line(mel_sym_str, 10.0, 15.0)

In [ ]:
# Read in text as words

# Create initial array of word starts
#   with initial guess of maximum error bars
# Create map of word -> word_index

In [ ]:
#Set #FILE: = #FILE:  guidetomen_02_rowland_64kb.mp3
#Set #OFFSET_START: = #OFFSET_START: 7.0
#Set #OFFSET_END: = #OFFSET_END: 613.0
offset_start, offset_end = 7.0, 613.0

In [ ]:
with open(mel_filename_test.replace('.melspectra.hkl', '.txt'), 'rt') as f:
    mel_txt = f.read()

txt_arr = mel_txt.replace('\n', ' ').split(' ')
txt_arr.insert(0, '#EOS') # Extra one at start
txt_arr.insert(0, '') # Helps start process
len(txt_arr), ','.join( txt_arr[0:10] )

In [ ]:
# Set up some matrices to fill in - these are timings in seconds 
txt_length_est = np.array( [ len(s) for s in txt_arr ] )
#txt_err = np.zeros_like( txt_starts )

In [ ]:
# Total up the lengths of the words (in characters) 
#  - the initial timing guess is going to be proportional 
txt_length_est[0:10]

In [ ]:
txt_starts = txt_length_est.cumsum()
txt_starts = offset_start + txt_starts*(offset_end-offset_start)/txt_starts[-1]

In [ ]:
txt_starts[0:10]

In [ ]:
txt_err_min = 5. # Seconds
txt_err_max = txt_starts[-1] * 0.10 # i.e. plus or minus the given percentage

In [ ]:
txt_err = np.array( [ (i-txt_starts.shape[0]) for i, txt in enumerate(txt_arr) ] )
txt_err = np.square(txt_err)
txt_err = txt_err.max() - txt_err # Now a parabola peaking in the middle

txt_err_scale = (txt_err_max - txt_err_min) / txt_err[ txt_starts.shape[0]//2 ]

txt_err = txt_err*txt_err_scale + txt_err_min

txt_err[0:5], txt_err[ txt_starts.shape[0]//2:txt_starts.shape[0]//2+5 ]

In [ ]:
## Global alignment

# Idea : Train up a word embedding based on range (within error bars)
# Function to map word to set of ranges
# Function to convert ranges into a % of whole
# Table of words with % coverage (to check whether that's a doable idea)
# Find word-range average vector (vs. not-in-word-range)

In [ ]:
word_to_idx={}
for i, w in enumerate(txt_arr):
    if w not in word_to_idx: word_to_idx[w]=[]
    word_to_idx[w].append( i )
word_to_idx['heart'], len(txt_arr), len(word_to_idx)  #bachelor mere

In [ ]:
def word_range_mask(w):
    mask = np.zeros_like(mel_sym)
    for i in word_to_idx.get(w, []):
        i_min = (txt_starts[i]-txt_err[i])/fft_step
        if i_min<0: i_min=0/fft_step
        i_next = i+1 if i<txt_starts.shape[0]-1 else i
        i_max = (txt_starts[i_next]+txt_err[i])/fft_step
        if i_max>mask.shape[0]: i_max=mask.shape[0]/fft_step
        #print("(i_min, i_max) = ", i_min, i_max)
        mask[ int(i_min):int(i_max) ] = 1.
    return mask

word_mask=word_range_mask('the')  # bachelor mere woman man the
np.sum(word_mask) / word_mask.shape[0]

In [ ]:
# Let's look at word frequencies, and mask coverage
words_freq_ordered = sorted(word_to_idx.keys(), key=lambda k: -len(word_to_idx[k]))
len(words_freq_ordered), words_freq_ordered[0], words_freq_ordered[-1]

In [ ]:
for w in words_freq_ordered:
    n = len(word_to_idx[w])
    if n<2: continue # Not enough for anything...
    word_mask=word_range_mask(w)
    coverage=np.sum(word_mask) / word_mask.shape[0]
    print("%6.2f%%, %4d, %s" % (coverage*100., n, w))

In [ ]:
# Now create histogram of symbol frequencies corresponding to mask
def histogram_freqs(mask, remove_silence=True):
    print(np.sum(mask) / mask.shape[0])
    #inside = bincount(mel_sym, weights=mask)
    n_sym = len(mel_sym_chars)
    
    inside_bins  = np.bincount(mel_sym, weights=mask)
    outside_bins = np.bincount(mel_sym, weights=1-mask)
    
    if remove_silence:
        inside_bins[mel_sym_dict[' ']]=0
        outside_bins[mel_sym_dict[' ']]=0
    
    rects1 = plt.bar(np.arange(0, n_sym)-.2, width=0.4, color='r',
                     height=inside_bins/np.sum(inside_bins))
    rects2 = plt.bar(np.arange(0, n_sym)+.2, width=0.4, color='b',
                     height=outside_bins/np.sum(outside_bins))

    plt.xlabel('Symbol#')
    plt.ylabel('Freq')
    plt.xticks(np.arange(0, n_sym, 1.0))
    plt.grid(True)
    plt.show()
    
histogram_freqs(word_range_mask('kiss'))

In [ ]:
# Let's find the ranges of the actual sounds in the 
# speech audio - ignoring short silences

silence_is_short_len = 8  # 100ms

spans, span_i, span_n = [], [], []
def add_span(span_start_index, s_i, s_n):
    if len(s_i)>0:
        spans.append(dict(
            t=span_start_index,
            span=s_i,
            count=s_n,
        ))
for idx, c in enumerate(mel_sym_cc):
    ci, cn = mel_sym_ci[idx], mel_sym_cn[idx]
    if ci==mel_sym_silence and cn>silence_is_short_len:
        add_span(mel_sym_ct[idx-1], span_i, span_n)
        span=[]
        continue # 
    span_i.append(ci)
    span_n.append(cn)
add_span(mel_sym_ct[idx-1], span_i, span_n)

len(spans), #spans[2]

In [ ]:
## Local alignment (looking within word-error-bar ranges only)

# Idea : Mostly textual alignment
# Have a look symbols in appropriate ranges for several word examples
# See whether a simple optimisation can align multiple segments
# Would reduce error bars massively
# Possibly : 
#   http://mlpy.sourceforge.net/docs/3.4/lcs.html#standard-lcs  (GPL3, though)
#   https://github.com/Samnsparky/py_common_subseq (MIT)
#   https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Longest_common_substring#Python_3 
#   https://docs.python.org/3/library/difflib.html (not quite...)

In [ ]:
#  !pip install py_common_subseq
#import py_common_subseq as subseq  # But doesn't output alignment

# Homebrew version : does what we actually want (return array of index correspondences)
def lcs(a_arr, b_arr):  
    m = np.zeros( (len(a_arr)+1, len(b_arr)+1) )
    # offset all the i and j by 1, since we need blank first row+col
    for i, a in enumerate(a_arr):
        for j, b in enumerate(b_arr):
            if a == b:
                m[i+1, j+1] = m[i, j] + 1
            else:
                m[i+1, j+1] = max(m[i+1, j], m[i, j+1])
                
    #?  a_i=np.zeros( (lengths[-1,-1], ))
    a_i, b_i = [], []
    i, j = len(a_arr), len(b_arr)
    while i>0 and j>0:
        if m[i, j] == m[i-1, j]:
            i -= 1
        elif m[i, j] == m[i, j-1]:
            j -= 1
        else: # a_arr[i-1] == b_arr[j-1]
            a_i.append(i-1)
            b_i.append(j-1)
            i -= 1
            j -= 1
    
    return a_i[::-1], b_i[::-1]

lcs([17, 9,99,7,4,8,3,7,5,2,4,1,2,1,2,4,5,6],
    [1, 17,11,7,4,8,3,7,0,2,4,0,2,  2,  5,6],)

In [ ]:
# http://colinraffel.com/publications/thesis.pdf

# Idea : Align symbols using linear DTW (v fast)
# Assign random increments to symbols, and see whether linear DTW can match the alignments
# https://blog.acolyer.org/2016/05/11/searching-and-mining-trillions-of-time-series-subsequences-under-dynamic-time-warping/

# Idea : Align (using DTW) the mels or embeddings within the word-error-bar segments
# This would be multiple small alignments too
# Needs vector DTW (like librosa has...)

#  https://github.com/pierre-rouanet/dtw (GPL3 : Unusable)
#  https://github.com/slaypni/fastdtw/tree/master/fastdtw  (MIT)
#  https://github.com/ricardodeazambuja/DTW (cardiod example : CC0 licensed)


## Combo

# Use global word embeddings to weight samples in local ranges


## Global alignment

# Idea : Train up a word embedding based on range (within error bars)
# Find word-range average vector (vs. not-in-word-range)
# Use this to do a DTW across all words vs all timesteps

# Alternative : Use same word embedding to do some kind of annealing :
#   gradually reducing error bars (and improving embedding, etc)